# Load Data and Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R
from preprocessing_utils import pickle_load, pickle_save

In [4]:
ellipse_cols = ['sat_id', 'ellipse_id', 'major', 'minor', 'loss', 'r00', 'r01', 'r02', 'r10', 'r11', 'r12', 'r20', 'r21', 'r22',
                't1', 't2', 't3']

In [5]:
data_sim = pd.read_csv('../data/data_sim.csv')
data_real = pd.read_csv('../data/data_real.csv')

In [6]:
data_sim = pd.DataFrame(data_sim.values, columns = ellipse_cols)
data_real = pd.DataFrame(data_real.values, columns = ellipse_cols)

In [7]:
data_sim.shape, data_real.shape

((38629, 17), (26790, 17))

# Axis Angle Representation

In [8]:
def convert_matrix(data):
    matrix = data.loc[:, 'r00':'r22'].values.reshape(-1, 3, 3)
    rotvec = np.array([R.from_matrix(m).as_rotvec() for m in matrix])
    data['r1'], data['r2'], data['r3'] = np.transpose(rotvec)
    data.drop(['r00', 'r01', 'r02', 'r10', 'r11', 'r12', 'r20', 'r21', 'r22'], axis=1, inplace=True)

In [9]:
convert_matrix(data_sim)
convert_matrix(data_real)

# Train-Test Segregation

In [10]:
merged_data = data_sim.merge(data_real, how='outer', on=['sat_id', 'ellipse_id'], suffixes=['_sim', '_real'], indicator=True)
merge = merged_data['_merge']
merged_data.drop(['ellipse_id', 'loss_sim', 'loss_real'], axis=1, inplace=True)

In [11]:
train_data = merged_data[merged_data['_merge'] == 'both'].drop('_merge', axis=1).reset_index(drop=True)
test_data = merged_data[merged_data['_merge'] == 'left_only'].drop('_merge', axis=1).dropna(axis=1).reset_index(drop=True)

In [12]:
sat_ids_train = train_data['sat_id']
sat_ids_test = test_data['sat_id']

In [13]:
train_data.drop('sat_id', axis=1, inplace=True)
test_data.drop('sat_id', axis=1, inplace=True)

In [14]:
labels = train_data.loc[:, 'major_real':]
train_data = train_data.loc[:, :'r3_sim']

In [15]:
cols = ['major', 'minor', 't1', 't2', 't3', 'r1', 'r2', 'r3']

In [16]:
train_data = pd.DataFrame(train_data.values, columns=cols)
test_data = pd.DataFrame(test_data.values, columns=cols)
labels = pd.DataFrame(labels.values, columns=cols)

In [17]:
#labels = labels - train_data

In [18]:
train_data.shape, test_data.shape, labels.shape

((26790, 8), (11839, 8), (26790, 8))

# Train-CV Split per Satellite

In [19]:
split_ratio = 0.1

In [20]:
train = {}
cv = {}
labels_train = {}
labels_cv = {}

In [21]:
for sat_id in range(600):
    sat_data = train_data[sat_ids_train == sat_id]
    sat_labels = labels[sat_ids_train == sat_id]
    train_size = int(sat_data.shape[0] * (1 - split_ratio))
    train[sat_id] = sat_data.iloc[:train_size, :].reset_index(drop=True)
    cv[sat_id] = sat_data.iloc[train_size:, :].reset_index(drop=True)
    labels_train[sat_id] = sat_labels.iloc[:train_size, :].reset_index(drop=True)
    labels_cv[sat_id] = sat_labels.iloc[train_size:, :].reset_index(drop=True)

# Network Architecture

In [23]:
class SatNet(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(SatNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 16)
        self.fc3 = nn.Linear(16, input_size)
    
    def forward(self, x):
        a1 = self.fc1(x) * torch.sigmoid(self.fc1(x))
        a2 = torch.tanh(self.fc2(a1))
        a3 = self.fc3(a2)
        return a3

In [34]:
def train_satnet(sat_data, satnet_params, iterations, lr):
    train, cv, labels_train, labels_cv = [torch.from_numpy(data).float() for data in sat_data]
    input_size, hidden_size = satnet_params
    model = SatNet(input_size, hidden_size)
    optim = torch.optim.Adam(model.parameters(), lr)
    
    for itr in range(iterations):
        optim.zero_grad()
        pred = model(train)
        pred_cv = model(cv)
        loss = torch.mean((pred - labels_train) ** 2)
        loss_cv = torch.mean((pred_cv - labels_cv) ** 2)
        loss.backward()
        optim.step()
        if(itr % 1000 == 0):
            print('Iteration: {} | Loss (Train): {} | Loss (CV): {}'.format(itr, loss.item(), loss_cv.item()))
        
    return model

In [35]:
input_size = train_data.shape[1]
hidden_size = 32
iterations = 10000
lr = 0.0001
satnet_params = [input_size, hidden_size]

In [36]:
models = {}
for sat_id in range(0):
    print('\nSatellited ID:', sat_id,'\n')
    sat_data = [train[sat_id].values, cv[sat_id].values, labels_train[sat_id].values, labels_cv[sat_id].values]
    models[sat_id] = train_satnet(sat_data, satnet_params, iterations, lr)


Satellited ID: 0 

Iteration: 0 | Loss (Train): 2.748077869415283 | Loss (CV): 2.7471532821655273
Iteration: 1000 | Loss (Train): 0.19799302518367767 | Loss (CV): 0.1924707442522049
Iteration: 2000 | Loss (Train): 0.011511015705764294 | Loss (CV): 0.004765321500599384
Iteration: 3000 | Loss (Train): 0.007106676697731018 | Loss (CV): 0.00039597373688593507
Iteration: 4000 | Loss (Train): 0.0070907375775277615 | Loss (CV): 0.0003892779059242457
Iteration: 5000 | Loss (Train): 0.0070821018889546394 | Loss (CV): 0.0003840412537101656
Iteration: 6000 | Loss (Train): 0.007065028417855501 | Loss (CV): 0.00037295627407729626
Iteration: 7000 | Loss (Train): 0.00701206224039197 | Loss (CV): 0.0003335139772389084
Iteration: 8000 | Loss (Train): 0.006923464592546225 | Loss (CV): 0.0002113649097736925
Iteration: 9000 | Loss (Train): 0.006915824953466654 | Loss (CV): 0.0001950891746673733

Satellited ID: 1 

Iteration: 0 | Loss (Train): 2.429593086242676 | Loss (CV): 1.752434253692627
Iteration: 10

In [73]:
models

{0: SatNet(
   (fc1): Linear(in_features=8, out_features=32, bias=True)
   (fc2): Linear(in_features=32, out_features=16, bias=True)
   (fc3): Linear(in_features=16, out_features=8, bias=True)
 ), 1: SatNet(
   (fc1): Linear(in_features=8, out_features=32, bias=True)
   (fc2): Linear(in_features=32, out_features=16, bias=True)
   (fc3): Linear(in_features=16, out_features=8, bias=True)
 ), 2: SatNet(
   (fc1): Linear(in_features=8, out_features=32, bias=True)
   (fc2): Linear(in_features=32, out_features=16, bias=True)
   (fc3): Linear(in_features=16, out_features=8, bias=True)
 ), 3: SatNet(
   (fc1): Linear(in_features=8, out_features=32, bias=True)
   (fc2): Linear(in_features=32, out_features=16, bias=True)
   (fc3): Linear(in_features=16, out_features=8, bias=True)
 ), 4: SatNet(
   (fc1): Linear(in_features=8, out_features=32, bias=True)
   (fc2): Linear(in_features=32, out_features=16, bias=True)
   (fc3): Linear(in_features=16, out_features=8, bias=True)
 ), 5: SatNet(
   (fc1)

In [76]:
satnets = pickle_load('../data/satnets.pickle')

In [78]:
satnets[0]

SatNet(
  (fc1): Linear(in_features=9, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=9, bias=True)
)